In [50]:
import pandas as pd

data=pd.read_csv('brain-data.csv')
data=data.drop(columns=['Unnamed: 0','mask_path'])
data['mask'].value_counts()
# df.head()

0    2556
1    1373
Name: mask, dtype: int64

In [51]:
data["mask"].replace({1: "Brain_tumor", 0: "Normal"}, inplace=True)
# data['mask'] = data['mask'].replace(['0'],'Normal')
data.head()

,image_path,mask
0,./lgg-mri-segmentation/kaggle_3m/TCGA_HT_7473_...,Brain_tumor
1,./lgg-mri-segmentation/kaggle_3m/TCGA_HT_7473_...,Brain_tumor
2,./lgg-mri-segmentation/kaggle_3m/TCGA_HT_7473_...,Normal
3,./lgg-mri-segmentation/kaggle_3m/TCGA_HT_7473_...,Normal
4,./lgg-mri-segmentation/kaggle_3m/TCGA_HT_7473_...,Normal


In [22]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.15)

In [11]:
# Write train and test CSV files
train.to_csv("./data/training/train.csv", index=False)
test.to_csv("./data/testing/test.csv", index=False)

In [16]:
import pandas as pd
train=pd.read_csv('./data/training/train.csv')
test=pd.read_csv('./data/testing/test.csv')


In [25]:
from keras_preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255., validation_split=0.1)

train_generator = datagen.flow_from_dataframe(train,
                                              directory='./data/training',
                                              x_col='image_path',
                                              y_col='mask',
                                              subset='training',
                                              class_mode='raw',
                                              batch_size=16,
                                              shuffle=True,
                                              target_size=(256,256)
                                                 )
valid_generator = datagen.flow_from_dataframe(train,
                                             directory='./data/training',
                                             x_col='image_path',
                                             y_col='mask',
                                             subset='validation',
                                             class_mode='raw',
                                             batch_size=16,
                                             shuffle=True,
                                             target_size=(256,256)
                                                 )
test_datagen = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagen.flow_from_dataframe(test,
                                                 directory='./data/testing',
                                                 x_col='image_path',
                                                 y_col='mask',
                                                 class_mode='raw',
                                                 batch_size=16,
                                                 shuffle=False,
                                                 target_size=(256,256)
                                                     )

Found 3006 validated image filenames.
Found 333 validated image filenames.
Found 590 validated image filenames.


In [26]:
 import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [28]:
prefix = 'brain-classification'
training_input_path   = sess.upload_data('./data', key_prefix=prefix+'/training')
# validation_input_path = sess.upload_data('./data/testing', key_prefix=prefix+'/validation')

In [30]:
print(training_input_path)
# print(validation_input_path)

s3://sagemaker-us-west-2-076039889373/brain-classification/training


In [45]:
 !pygmentize train.py

import argparse, os
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.optimizers import SGD
from keras.utils import multi_gpu_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import *
from tensorflow.python.keras import Sequential
from tensorflow.keras import layers, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
from keras.optimizers import SGD

if __name__ == '__main__':
    parser = argparse.ArgumentParser()


    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--tra

In [46]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='train.py', 
                          role=role,
                          instance_count=1, 
                          instance_type='ml.m4.xlarge',
                          framework_version='1.12', 
                          py_version='py3',
                          script_mode=True)

In [47]:
tf_estimator.hyperparameters()

{'model_dir': 'null'}

In [48]:
tf_estimator.fit(training_input_path)

2022-05-29 14:14:19 Starting - Starting the training job...
2022-05-29 14:14:47 Starting - Preparing the instances for trainingProfilerReport-1653833659: InProgress
.........
2022-05-29 14:16:03 Downloading - Downloading input data...............
2022-05-29 14:18:44 Training - Training image download completed. Training in progress.2022-05-29 14:18:38,007 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2022-05-29 14:18:38,013 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-05-29 14:18:38,587 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-05-29 14:18:38,611 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-05-29 14:18:38,635 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "training": "/opt/ml/input/data/training"
    },
    "current_host": "algo-1",
 

# BUILDING A SEGMENTATION MODEL TO LOCALIZE TUMOR

In [52]:
brain_df_mask = data[data['mask'] == "Brain_tumor"]
brain_df_mask.shape

(1373, 2)

In [53]:
# creating test, train and val sets
X_train, X_val = train_test_split(brain_df_mask, test_size=0.15)
X_test, X_val = train_test_split(X_val, test_size=0.5)
print("Train size is {}, valid size is {} & test size is {}".format(len(X_train), len(X_val), len(X_test)))

train_ids = list(X_train.image_path)
train_mask = list(X_train.mask_path)

val_ids = list(X_val.image_path)
val_mask= list(X_val.mask_path)

Train size is 1167, valid size is 103 & test size is 103


AttributeError: 'DataFrame' object has no attribute 'mask_path'